In [ ]:
import grid2op
import numpy as np
from superposition_theorem import State
env = grid2op.make("l2rpn_case14_sandbox")
param = env.parameters
param.ENV_DC = True  # force the computation of the powerflow in DC mode
param.MAX_LINE_STATUS_CHANGED = 99999
param.MAX_SUB_CHANGED = 99999
param.NO_OVERFLOW_DISCONNECTION = True
env.change_parameters(param)
env.change_forecast_parameters(param)

# Line disconnection

Here we use the "State" interface to compute the disconnection (at its creation) and then we can see the results of each n-k (which are a lot)

First, let's initialize the "State" with the unary line disconnections we want to combine

In [ ]:
obs = env.reset()
state = State.from_grid2op_obs(obs, line_ids_disc_unary=(1, 2, 3, 4, 7))

And now we can compute the flows after the disconnection of 2 powerlines (for example): 

In [ ]:
p_or_12 = state.compute_flows_disco_lines((1, 2))

And now we can check the results compared to using a real simulator:

In [ ]:
obs_true, *_ = obs.simulate(env.action_space({"set_bus": {"lines_or_id": [(1, -1), (2, -1)]}}), time_step=0)
por_true = obs_true.p_or
print(f"Max difference: {np.abs(p_or_12 - por_true).max():.2e} MW")

# Line reconnection

In this second part, we show how we could reconnect powerline. For this, we first generate a grid state with some line disconnected (1 and 2 for the example) and then see how we can leverage the superposition theorem to reconnect it.

In [ ]:
obs = env.reset()
start_obs, *_  = env.step(env.action_space({"set_bus": {"lines_or_id": [(1, -1), (2, -1)]}}))
state = State.from_grid2op_obs(start_obs, line_ids_reco_unary=(1, 2))

Now we can reconnect these powerlines using the superposition theorem:

In [ ]:
p_or_12 = state.compute_flows_reco_lines((1, 2))

And finally, we can check the results

In [ ]:
obs_true, *_ = start_obs.simulate(env.action_space({"set_bus": {"lines_or_id": [(1, 1), (2, 1)]}}), time_step=0)
por_true = obs_true.p_or
print(f"Max difference: {np.abs(p_or_12 - por_true).max():.2e} MW")

# Topology modification (node splitting)

Work in progress :-)